In [6]:
pip install kafka-python -i https://pypi.douban.com/simple

Looking in indexes: https://pypi.douban.com/simple
     |████████████████████████████████| 235kB 1.2MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [36]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
from kafka import KafkaProducer
from kafka import KafkaConsumer
from kafka.errors import KafkaError
import json



def gendata(i,imgname1,imgname2):
    rec={
        "FaceListObject": {
            "Type": "",
            "FaceObject": [
                {
                    "FaceID": "31011577011218000105022020030314135900001",
                    "GenderCode": "",
                    "FaceAppearTime": "20200303141359",
                    "DeviceID": "31011577011218000105",
                    "SourceID": "1159077405",
                    "IsVictim": "1",
                    "SubImageList": {
                        "SubImageInfoObject": [
                            {
                                "Type": "11",
                                "StoragePath": "/ZNVFSIMG/group1/M00/00/00/CkhCvl5d9aiAfG8yAAA7NFaBKyk841.jpg",
                                "DeviceID": "31011577011218000105",
                                "ImageID": "",
                                "EventSort": "1",
                                "Data": "",
                                "Height": "0",
                                "ShotTime": "20200303141359",
                                "FileFormat": "",
                                "Width": "0"
                            },
                            {
                                "Type": "14",
                                "StoragePath": "/ZNVFSIMG/group1/M00/00/00/CkhCvl5d9aiAf3xBAAUryvbxCaY728.jpg",
                                "DeviceID": "31011577011218000105",
                                "ImageID": "",
                                "EventSort": "1",
                                "Data": "",
                                "Height": "0",
                                "ShotTime": "20200303141359",
                                "FileFormat": "",
                                "Width": "0"
                            }
                        ]
                    },
                    "LeftTopY": "0",
                    "LeftTopX": "0",
                    "FaceDisAppearTime": "20200303141359",
                    "IsSuspectedTerrorist": "0",
                    "IsDriver": "1",
                    "IsForeigner": "0",
                    "InfoKind": "1",
                    "RightBtmY": "0",
                    "IsCriminalInvolved": "0",
                    "RightBtmX": "0",
                    "IsDetainees": "0",
                    "IsSuspiciousPerson": "1",
                    "LocationMarkTime": "20200303141359",
                    "IDNumber": ""
                }
            ]
        }
    }
    
    devid="3101157701121800010%d"%(i%10)
    
    obj=rec['FaceListObject']['FaceObject'][0]
    obj['DeviceID']=devid
    obj["FaceID"]=devid+"022020030314135900%03d"%i
    shorttime=140000+i//60*100+i%60
    obj["FaceAppearTime"]="20200303"+str(shorttime)
    obj["FaceDisAppearTime"]="20200303"+str(shorttime+3)
    obj["LocationMarkTime"]=obj['FaceAppearTime']
    subimg=obj['SubImageList']['SubImageInfoObject']
    subimg[0]['StoragePath']='/datascience/'+imgname1
    subimg[1]['StoragePath']='/datascience/'+imgname2
    subimg[0]['DeviceID']=devid
    subimg[1]['DeviceID']=devid
    subimg[0]['ShotTime']=shorttime
    subimg[1]['ShotTime']=shorttime
    
    return rec


In [37]:


##测试生产模块
producer = KafkaProducer(bootstrap_servers=['10.45.154.209:9092'])

seq=123
rec=gendata(seq,"img1.jpg","img2.jpg")
devid="3101157701121800010%d"%(seq%10)
print("sending...")
#xx=producer.send("gangertest",key= bytes(devid, encoding = "utf8") ,value=json.dumps(rec).encode('utf-8'))
xx=producer.send("gangertest",key= b'aaaaa' ,value=b'bbbbbbbbb')
result = xx.get(timeout= 100)
print(result)
print("finish")
#for id in range(10):
#    params = '{abetst}:{null}---'+str(i)
#    producer.sendjsondata(params)
##测试消费模块
#消费模块的返回格式为ConsumerRecord(topic=u'ranktest', partition=0, offset=202, timestamp=None, 
#\timestamp_type=None, key=None, value='"{abetst}:{null}---0"', checksum=-1868164195, 
#\serialized_key_size=-1, serialized_value_size=21)
#consumer = Kafka_consumer('127.0.0.1', 9092, "ranktest", 'test-python-ranktest')
#message = consumer.consume_data()
#for i in message:
#    print(i.value)




sending...


KafkaTimeoutError: KafkaTimeoutError: Batch for TopicPartition(topic='gangertest', partition=2) containing 1 record(s) expired: 30 seconds have passed since batch creation plus linger time

In [28]:
seq=123
rec=gendata(seq,"img1.jpg","img2.jpg")
devid="3101157701121800010%d"%(seq%10)
json.dumps(rec).encode('utf-8')

b'{"FaceListObject": {"Type": "", "FaceObject": [{"FaceID": "31011577011218000103022020030314135900123", "GenderCode": "", "FaceAppearTime": "20200303140203", "DeviceID": "31011577011218000103", "SourceID": "1159077405", "IsVictim": "1", "SubImageList": {"SubImageInfoObject": [{"Type": "11", "StoragePath": "/datascience/img1.jpg", "DeviceID": "31011577011218000103", "ImageID": "", "EventSort": "1", "Data": "", "Height": "0", "ShotTime": 140203, "FileFormat": "", "Width": "0"}, {"Type": "14", "StoragePath": "/datascience/img2.jpg", "DeviceID": "31011577011218000103", "ImageID": "", "EventSort": "1", "Data": "", "Height": "0", "ShotTime": 140203, "FileFormat": "", "Width": "0"}]}, "LeftTopY": "0", "LeftTopX": "0", "FaceDisAppearTime": "20200303140206", "IsSuspectedTerrorist": "0", "IsDriver": "1", "IsForeigner": "0", "InfoKind": "1", "RightBtmY": "0", "IsCriminalInvolved": "0", "RightBtmX": "0", "IsDetainees": "0", "IsSuspiciousPerson": "1", "LocationMarkTime": "20200303140203", "IDNumbe

In [40]:
import os
imglist=os.listdir("../../../input/capshot3000/")
imgmap={}
for img in imglist:
    name=img[:-4]
    if len(name)>15:
        sname=name.split('_')
        smallimg=name
        bigimg='_'.join(sname[:3])
        imgmap[smallimg]=bigimg
print(imgmap)

{'FD_100_22142644_207_32364640': 'FD_100_22142644', 'FD_100_27132426_238_34658416': 'FD_100_27132426', 'FD_100_5142648_182_34266128': 'FD_100_5142648', 'FD_100_6125021_871_39640200': 'FD_100_6125021', 'FD_100_919298_190_33413216': 'FD_100_919298', 'FD_101_2214274_209_32364640': 'FD_101_2214274', 'FD_101_27132517_239_34658416': 'FD_101_27132517', 'FD_101_5142656_183_34266128': 'FD_101_5142656', 'FD_101_6125031_874_39640200': 'FD_101_6125031', 'FD_101_919301_191_33413216': 'FD_101_919301', 'FD_102_2214289_210_32364640': 'FD_102_2214289', 'FD_102_27132522_241_34658416': 'FD_102_27132522', 'FD_102_5142658_184_34266128': 'FD_102_5142658', 'FD_102_6125041_881_39640200': 'FD_102_6125041', 'FD_102_919303_193_33413216': 'FD_102_919303', 'FD_103_22143221_213_32364640': 'FD_103_22143221', 'FD_103_2713274_244_34658416': 'FD_103_2713274', 'FD_103_514271_185_34266128': 'FD_103_514271', 'FD_103_6125130_882_39640200': 'FD_103_6125130', 'FD_103_9193011_195_33413216': 'FD_103_9193011', 'FD_104_2214365_2

In [56]:
import time
'%s%d'%('sdf',123)

'sdf123'